In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#importing the necessary library
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


In [2]:
#Importing necessary libraries
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Replace sparse with sparse_output
encoder = OneHotEncoder(sparse_output=True)

#loading the data frames and assigning names
df_train = pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")
df_sub =pd.read_csv("/kaggle/input/playground-series-s4e6/sample_submission.csv")
df_train.info()

df_train = df_train.drop('id',axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [3]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51012 entries, 0 to 51011
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              51012 non-null  int64  
 1   Marital status                                  51012 non-null  int64  
 2   Application mode                                51012 non-null  int64  
 3   Application order                               51012 non-null  int64  
 4   Course                                          51012 non-null  int64  
 5   Daytime/evening attendance                      51012 non-null  int64  
 6   Previous qualification                          51012 non-null  int64  
 7   Previous qualification (grade)                  51012 non-null  float64
 8   Nacionality                                     51012 non-null  int64  
 9   Mother's qualification                 

In [4]:
# Keep rows 1 to 3 (exclusive of row 3, i.e., rows 1 and 2)
df_reduced = df_train.iloc[1:76518]
df_reduced.info()
df_reduced.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76517 entries, 1 to 76517
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  76517 non-null  int64  
 1   Application mode                                76517 non-null  int64  
 2   Application order                               76517 non-null  int64  
 3   Course                                          76517 non-null  int64  
 4   Daytime/evening attendance                      76517 non-null  int64  
 5   Previous qualification                          76517 non-null  int64  
 6   Previous qualification (grade)                  76517 non-null  float64
 7   Nacionality                                     76517 non-null  int64  
 8   Mother's qualification                          76517 non-null  int64  
 9   Father's qualification                 

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
5,1,39,1,171,1,19,133.1,1,19,19,...,0,0,0,0,0.000000,0,13.9,-0.3,0.79,Dropout


In [5]:
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder

# Assuming df_reduced is your dataframe and 'Target' is the column to encode
label_encoder = LabelEncoder()


# Using .loc to explicitly modify the original dataframe
df_reduced.loc[:, 'Category_LabelEncoded'] = label_encoder.fit_transform(df_reduced['Target'])


/tmp/ipykernel_18/3475420404.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced.loc[:, 'Category_LabelEncoded'] = label_encoder.fit_transform(df_reduced['Target'])


In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

X = df_reduced.drop(['Target','Category_LabelEncoded'],axis=1)
y = df_reduced['Category_LabelEncoded'] 

y

1        0
2        0
3        1
4        2
5        0
        ..
76513    2
76514    2
76515    1
76516    0
76517    2
Name: Category_LabelEncoded, Length: 76517, dtype: int64

In [7]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Instantiate GradientBoostingClassifier
gbc = GradientBoostingClassifier()

# Fit the model
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [9]:
# Make predictions
y_pred = gbc.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8231834814427601


In [10]:
df_test = df_test.drop('id',axis=1)

In [11]:
# Make predictions
y_pred = gbc.predict(df_test)
y_pred

array([0, 2, 2, ..., 0, 0, 0])

In [12]:
# Define mapping from integers to categorical labels
mapping = {
    0: 'Dropout',
    1: 'Enrolled',
    2: 'Graduate'
}

# Convert integer array to categorical labels
categorical_array = [mapping[val] for val in y_pred]
# Convert NumPy array to DataFrame
df = pd.DataFrame(categorical_array)

In [13]:
merged_df = pd.merge(df_sub, df, left_index=True, right_index=True)
df_up = merged_df.drop('Target',axis=1)
df_submit = merged_df.rename(columns={'0':'Target'}, inplace=True)


In [14]:
df_submit

In [15]:
df_sub

,id,Target
0,76518,Graduate
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Graduate
...,...,...
51007,127525,Graduate
51008,127526,Graduate
51009,127527,Graduate
51010,127528,Graduate
